**Transferencia de aprendizaje aplicado a perros y gatos\**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import shutil

In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
import os
# Ver qué hay en tu Drive
print(os.listdir('/content/drive/MyDrive/'))

['Classroom', 'GUIA_EXCOBA.pdf', 'CERTIFICADO_427740.pdf', 'fisicaserway_Vol1.pdf', 'Tarea 1.pdf', 'Práctica 5.pdf', 'Conceptos de física que considero importantes.pdf', 'comprobanteVigenciaDerechos46220438819 (1).pdf', 'comprobanteVigenciaDerechos46220438819 (2).pdf', 'Práctica 10 energía.pdf', 'Práctica 11 recipientes.pdf', 'Práctica 12 elementos de un circuito eléctico.pdf', 'Colab Notebooks', 'Práctica final.pdf', 'Jan 10, 1.26 PM\u200b.aac', 'Dancing with Qubits How quantum computing works and how it can change the world (Robert S. Sutor) (z-lib.org).pdf', 'Carátula IMSS.pdf', 'Jan 19, 7.16 PM\u200b.aac', 'Jan 29, 2.25 PM\u200b.aac', 'Feb 1, 3.11 PM\u200b.aac', 'INE_MAMA.pdf', 'INE_OscarR_Sánchez.pdf', 'Reporte 1.gdoc', 'Documento sin título (1).gdoc', 'Práctica 3 laboratorio de mecánica.gdoc', 'Práctica 4 laboratorio de mecánica..gdoc', 'Práctica 5 laboratorio de mecánica.gdoc', 'CERTIFICADO_BACHILLERATO.pdf', 'Pago_Constancia.pdf', 'CONSTANCIA DE ESTUDIOS (3).pd

In [19]:
# Verificamos la estructura del dataset

train_dir = '/content/drive/MyDrive/Train'

print("Verificando contenido de la carpeta Train:")
items = os.listdir(train_dir)
print(f"Total de items: {len(items)}\n")

# Mostrar primeros 10 items
for item in items[:10]:
    item_path = os.path.join(train_dir, item)
    if os.path.isdir(item_path):
        num_files = len(os.listdir(item_path))
        print(f"  📁 {item}: {num_files} archivos")
    else:
        print(f"  📄 {item}")

print("\n✓ Verificación completa")

Verificando contenido de la carpeta Train:
Total de items: 4340

  📄 dog.8984.jpg
  📄 cat.11502.jpg
  📄 dog.12346.jpg
  📄 cat.2055.jpg
  📄 dog.6765.jpg
  📄 cat.4430.jpg
  📄 dog.9456.jpg
  📄 dog.8990.jpg
  📄 cat.11516.jpg
  📄 cat.10608.jpg

✓ Verificación completa


In [22]:
# Copiar la carpeta Train de Drive a Colab
source = '/content/drive/MyDrive/Train'
destination = '/content/Train'

print("Copiando dataset a Colab (esto puede tardar unos minutos)...")
shutil.copytree(source, destination)
print(f"✓ Dataset copiado a {destination}")

# Ahora trabajamos con esta ruta local
train_dir = '/content/Train'

Copiando dataset a Colab (esto puede tardar unos minutos)...
✓ Dataset copiado a /content/Train


In [23]:
# PASO 3: Organizar imágenes en subcarpetas Cat y Dog

train_dir = '/content/Train'

# Crear carpetas Cat y Dog
cat_dir = os.path.join(train_dir, 'Cat')
dog_dir = os.path.join(train_dir, 'Dog')

os.makedirs(cat_dir, exist_ok=True)
os.makedirs(dog_dir, exist_ok=True)

# Mover imágenes según su nombre
cat_count = 0
dog_count = 0

for filename in os.listdir(train_dir):
    file_path = os.path.join(train_dir, filename)

    # Saltar si es una carpeta
    if os.path.isdir(file_path):
        continue

    # Mover según el prefijo del nombre
    if filename.lower().startswith('cat'):
        shutil.move(file_path, os.path.join(cat_dir, filename))
        cat_count += 1
    elif filename.lower().startswith('dog'):
        shutil.move(file_path, os.path.join(dog_dir, filename))
        dog_count += 1

print(f"Imágenes reorganizadas:")
print(f"  - Gatos: {cat_count} imágenes")
print(f"  - Perros: {dog_count} imágenes")

Imágenes reorganizadas:
  - Gatos: 2174 imágenes
  - Perros: 2166 imágenes


In [25]:
# Cargamos el dataset con Tensorflow
import tensorflow as tf

train_dir = '/content/Train'

# Parámetros
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

# Cargar dataset de entrenamiento (80%)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Cargar dataset de validación (20%)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    shuffle=True,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Verificar las clases
class_names = train_dataset.class_names
print(f"\n✓ Clases detectadas: {class_names}")
print(f"✓ Batches de entrenamiento: {len(train_dataset)}")
print(f"✓ Batches de validación: {len(validation_dataset)}")

Found 4340 files belonging to 2 classes.
Using 3472 files for training.
Found 4340 files belonging to 2 classes.
Using 868 files for validation.

✓ Clases detectadas: ['Cat', 'Dog']
✓ Batches de entrenamiento: 109
✓ Batches de validación: 28


In [26]:
# Optimizar el rendimiento
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

print("✓ Datasets optimizados y listos para entrenar")

✓ Datasets optimizados y listos para entrenar
